In [1]:
import warnings
warnings.filterwarnings("ignore")
import string
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
from gtts import gTTS
import playsound
import os
import json

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
#from keras.optimizers import 
import random
from sklearn.model_selection import train_test_split
from keras.models import load_model
import speech_recognition as sr


playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.


In [3]:
words = []
classes = []
docs = []
ignore_words = string.punctuation
data_file = open("intents.json").read()
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        docs.append((w, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [6]:
words

["'s",
 '...',
 'able',
 'about',
 'an',
 'appreciate',
 'are',
 'assignment',
 'awesome',
 'batch',
 'be',
 'between',
 'bot',
 'bye',
 'can',
 'chatting',
 'cost',
 'could',
 'course',
 'day',
 'detail',
 'do',
 'doe',
 'error',
 'find',
 'for',
 'get',
 'go',
 'good',
 'goodbye',
 'great',
 'have',
 'help',
 'helpful',
 'hey',
 'hi',
 'hmm',
 'hola',
 'how',
 'i',
 'in',
 'ineuron',
 'is',
 'issue',
 'it',
 'join',
 'know',
 'later',
 'me',
 'meet',
 'more',
 "n't",
 'name',
 'next',
 'nice',
 'not',
 'of',
 'price',
 'pricing',
 'problem',
 'provide',
 'recording',
 'see',
 'should',
 'showing',
 'solve',
 'solved',
 'start',
 'still',
 'submit',
 'sup',
 'tell',
 'thank',
 'thanks',
 'that',
 'the',
 'there',
 'think',
 'this',
 'till',
 'time',
 'to',
 'use',
 'what',
 'when',
 'where',
 'who',
 'with',
 'working',
 'yo',
 'you',
 'your']

In [7]:
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

In [8]:
training = []
output_empty = [0] * len(classes)

for doc in docs:
    bag = []
    pat_words = doc[0]

    pat_words = [lemmatizer.lemmatize(word.lower()) for word in pat_words]

    for w in words:
        bag.append(1) if w in pat_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

In [9]:
X_train = list(training[:,0])
Y_train = list(training[:,1])
#X_train, x_test, Y_train, y_test = train_test_split(list(training[:,0]), list(training[:,1]), test_size=0.2)

In [10]:
model = Sequential()
model.add(Dense(128, input_shape = (len(X_train[0]),), activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(len(Y_train[0]), activation = 'softmax'))

model.compile(optimizer = 'SGD', loss = 'categorical_crossentropy', metrics = ['accuracy'])
hist = model.fit(np.array(X_train), np.array(Y_train), epochs = 200, batch_size = 5, verbose = 1)

model.save("chatbot.h5", hist)

Epoch 1/200


2022-06-26 13:58:37.038120: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


12/12 [==============================] - 0s 2ms/step - loss: 2.5133 - accuracy: 0.2105
Epoch 2/200
12/12 [==============================] - 0s 2ms/step - loss: 2.5278 - accuracy: 0.0702
Epoch 3/200
12/12 [==============================] - 0s 2ms/step - loss: 2.5056 - accuracy: 0.1228
Epoch 4/200
12/12 [==============================] - 0s 2ms/step - loss: 2.4750 - accuracy: 0.1053
Epoch 5/200
12/12 [==============================] - 0s 2ms/step - loss: 2.4909 - accuracy: 0.1404
Epoch 6/200
12/12 [==============================] - 0s 2ms/step - loss: 2.4691 - accuracy: 0.1404
Epoch 7/200
12/12 [==============================] - 0s 1ms/step - loss: 2.4610 - accuracy: 0.1754
Epoch 8/200
12/12 [==============================] - 0s 2ms/step - loss: 2.4314 - accuracy: 0.2456
Epoch 9/200
12/12 [==============================] - 0s 2ms/step - loss: 2.4423 - accuracy: 0.1930
Epoch 10/200
12/12 [==============================] - 0s 2ms/step - loss: 2.4165 - accuracy: 0.1754
Epoch 11/200
12/12 [=

12/12 [==============================] - 0s 2ms/step - loss: 1.3115 - accuracy: 0.7368
Epoch 84/200
12/12 [==============================] - 0s 2ms/step - loss: 1.4114 - accuracy: 0.6491
Epoch 85/200
12/12 [==============================] - 0s 2ms/step - loss: 1.3850 - accuracy: 0.5614
Epoch 86/200
12/12 [==============================] - 0s 1ms/step - loss: 1.3232 - accuracy: 0.6842
Epoch 87/200
12/12 [==============================] - 0s 2ms/step - loss: 1.2977 - accuracy: 0.7368
Epoch 88/200
12/12 [==============================] - 0s 2ms/step - loss: 1.2533 - accuracy: 0.7193
Epoch 89/200
12/12 [==============================] - 0s 2ms/step - loss: 1.2306 - accuracy: 0.6667
Epoch 90/200
12/12 [==============================] - 0s 2ms/step - loss: 1.3111 - accuracy: 0.6842
Epoch 91/200
12/12 [==============================] - 0s 2ms/step - loss: 1.2744 - accuracy: 0.7368
Epoch 92/200
12/12 [==============================] - 0s 2ms/step - loss: 1.1582 - accuracy: 0.7719
Epoch 93/200


12/12 [==============================] - 0s 2ms/step - loss: 0.5798 - accuracy: 0.8421
Epoch 165/200
12/12 [==============================] - 0s 2ms/step - loss: 0.4326 - accuracy: 0.9298
Epoch 166/200
12/12 [==============================] - 0s 2ms/step - loss: 0.5674 - accuracy: 0.8596
Epoch 167/200
12/12 [==============================] - 0s 2ms/step - loss: 0.4716 - accuracy: 0.8947
Epoch 168/200
12/12 [==============================] - 0s 2ms/step - loss: 0.5341 - accuracy: 0.9123
Epoch 169/200
12/12 [==============================] - 0s 2ms/step - loss: 0.5072 - accuracy: 0.8596
Epoch 170/200
12/12 [==============================] - 0s 2ms/step - loss: 0.4749 - accuracy: 0.9298
Epoch 171/200
12/12 [==============================] - 0s 2ms/step - loss: 0.3915 - accuracy: 0.9474
Epoch 172/200
12/12 [==============================] - 0s 2ms/step - loss: 0.5173 - accuracy: 0.8421
Epoch 173/200
12/12 [==============================] - 0s 2ms/step - loss: 0.5028 - accuracy: 0.8947
Epoc

In [11]:
saved_model = load_model("chatbot.h5")
saved_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               11904     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 12)                780       
                                                                 
Total params: 20,940
Trainable params: 20,940
Non-trainable params: 0
_________________________________________________________________


In [12]:
def clean_up_sequence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [13]:
def bow(sentence, words, show_details = True):
    sentence_words = clean_up_sequence(sentence)

    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("Found in bag: %s" % w)
    return(np.array(bag))

In [14]:
def predict_class(sentence, model):
    p = bow(sentence, words, show_details = False)
    res = model.predict(np.array([p]))[0]
    error = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>error]

    results.sort(key = lambda x: x[1], reverse = True)
    return_list = []

    for r in results:
        return_list.append({"intent": classes[r[0]], "probability":str(r[1])})
    
    return return_list

In [15]:
def getResponse(ints, intents_json):
    
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

# def chat(dial):
#     return chatbot_response(dial)

In [16]:
def voice_input():
    r = sr.Recognizer()
    
# using microphone as source for input
    with sr.Microphone() as source:
        print("Silence Please, Calibrating...")
        r.adjust_for_ambient_noise(source, duration=1)
        print("Calibrated, Please Say Something!")

        # listens for the user's input
        audio = r.listen(source)

        # using google to recognize the speech
        MyText = r.recognize_google(audio)
        MyText = MyText.lower()
        print(MyText)
        return MyText


In [17]:
def speak(temp):
    voice = gTTS(text=temp, lang="en")
    voice.save("temp.mp3")
    playsound.playsound("temp.mp3")
    os.remove("temp.mp3")

In [18]:
def start_chat():
    print("KIRA: Hello! How can I help you?\n\n")
    while True:
        inp = voice_input().lower()
        if inp.lower() == "end":
            break
        if inp.lower() == '' or inp.lower() == '*':
            print("Please re-type your query")
            print("-"*50)
        else:
            resp = str(chatbot_response(inp))
            print(f"KIRA: {resp}" + '\n')
            speak(resp)
            print("-" * 50)


In [19]:
start_chat()

KIRA: Hello! How can I help you?


Silence Please, Calibrating...


ALSA lib pcm_dsnoop.c:638:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1075:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_dmix.c:1075:(snd_pcm_dmix_open) unable to open slave


Calibrated, Please Say Something!
hai
1/1 [==============================] - 0s 74ms/step
KIRA: Good to see you

--------------------------------------------------
Silence Please, Calibrating...
Calibrated, Please Say Something!
birbal
1/1 [==============================] - 0s 14ms/step
KIRA: Hello!

--------------------------------------------------
Silence Please, Calibrating...
Calibrated, Please Say Something!
what is a neurone
1/1 [==============================] - 0s 26ms/step
KIRA: I am KIRA Bot

--------------------------------------------------
Silence Please, Calibrating...
Calibrated, Please Say Something!
neurone
1/1 [==============================] - 0s 27ms/step
KIRA: How may I help you?

--------------------------------------------------
Silence Please, Calibrating...
Calibrated, Please Say Something!
which course should i buy
1/1 [==============================] - 0s 27ms/step
KIRA: Please Visit https://courses.ineuron.ai/ for more information about courses

-----------

KeyboardInterrupt: 